In [1]:
import os
from pathlib import Path
from typing import List, Set, Optional

def concatenate_files_with_separator(
    root_directory: str,
    output_filepath: str,
    target_extensions: List[str],
    excluded_folders: Optional[Set[str]] = None
) -> None:
    """
    读取目录下指定扩展名的文件内容，并使用路径/文件名作为分隔符，
    将它们拼接到一个输出 TXT 文件中。支持排除某些文件夹。

    Args:
        root_directory (str): 要搜索的根目录路径。
        output_filepath (str): 输出 TXT 文件的路径。
        target_extensions (List[str]): 要读取的文件扩展名列表 (例如: ['.txt', '.py'])。
                                       确保扩展名以 '.' 开头。
        excluded_folders (Optional[Set[str]]): 要排除的文件夹名称集合 (例如: {'__pycache__', '.venv'})。
                                               这些文件夹及其子文件夹下的文件将被忽略。
                                               默认为 None (不排除任何文件夹)。
    """
    if excluded_folders is None:
        excluded_folders = set()

    root_path = Path(root_directory).resolve()
    output_file = Path(output_filepath).resolve()

    if not root_path.is_dir():
        print(f"错误: 根目录 '{root_directory}' 不存在或不是一个目录。")
        return

    # 确保目标扩展名以 '.' 开头
    processed_extensions = [ext if ext.startswith('.') else f".{ext}" for ext in target_extensions]
    if not processed_extensions:
        print("错误: 必须提供至少一个目标文件扩展名。")
        return

    print(f"开始处理目录: {root_path}")
    print(f"输出文件: {output_file}")
    print(f"目标扩展名: {processed_extensions}")
    print(f"排除的文件夹: {excluded_folders if excluded_folders else '无'}")

    file_count = 0
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for dirpath_str, dirnames, filenames in os.walk(root_path, topdown=True):
            current_dir_path = Path(dirpath_str)

            # 排除文件夹：修改 dirnames 列表以阻止 os.walk 进入这些目录
            # 这是 os.walk 的一个特性，在 topdown=True 时有效
            dirnames[:] = [d for d in dirnames if d not in excluded_folders]

            # 检查当前目录本身是否在排除列表中（这主要针对根目录直接是排除目录的情况，
            # 或者 excluded_folders 包含像 'src/tests' 这样的相对路径，但我们这里用的是文件夹名）
            # 对于简单的文件夹名排除，上面的 dirnames[:] 修改已经足够了。
            # 如果 current_dir_path.name 在 excluded_folders 中，我们就不应该处理这个目录中的文件
            # 但由于上面修改了 dirnames，os.walk 不会再进入这些目录的子目录。
            # 我们只需要确保不处理当前这个被排除目录下的文件。
            if current_dir_path.name in excluded_folders and current_dir_path != root_path:
                 print(f"  跳过排除的文件夹中的文件: {current_dir_path}")
                 continue


            for filename in filenames:
                file_path = current_dir_path / filename
                if file_path.suffix.lower() in processed_extensions:
                    relative_path = file_path.relative_to(root_path)
                    # 使用 POSIX 风格的路径分隔符 '/' 以保持一致性
                    separator = f"--- START: {relative_path.as_posix()} ---"
                    end_separator = f"--- END: {relative_path.as_posix()} ---\n\n"

                    outfile.write(separator + "\n")
                    try:
                        with open(file_path, 'r', encoding='utf-8', errors='ignore') as infile:
                            outfile.write(infile.read())
                        outfile.write("\n" + end_separator) # 在文件内容后加换行，然后是结束分隔符
                        file_count += 1
                        print(f"  已处理: {relative_path.as_posix()}")
                    except Exception as e:
                        error_message = f"无法读取文件 '{file_path.as_posix()}': {e}"
                        outfile.write(f"[错误: {error_message}]\n")
                        outfile.write(end_separator)
                        print(f"  错误: {error_message}")

    if file_count > 0:
        print(f"\n处理完成！{file_count} 个文件的内容已拼接到 '{output_file.as_posix()}'。")
    else:
        print(f"\n处理完成。在 '{root_path.as_posix()}' 中未找到指定扩展名的文件 (或均在排除文件夹内)。")
        if not output_file.exists() or output_file.stat().st_size == 0 :
             print(f"输出文件 '{output_file.as_posix()}' 为空或未创建。")


if __name__ == '__main__':
    # ----- 配置参数 -----
    # 1. 要搜索的根目录
    source_directory = "D:\ALL IN AI\IDTAgent\IDT_Agent_Native"  # 请替换为你的实际目录

    # 2. 输出的 TXT 文件名和路径
    concatenated_output_file = "completeCode.txt"

    # 3. 要包含的文件扩展名 (例如: '.txt', '.py', '.md')
    include_extensions = ['.txt', '.py', '.js', 'html','css']  # 请根据需要修改

    # 4. 要排除的文件夹名称 (例如: '__pycache__', 'node_modules', '.git')
    #    这些文件夹及其所有子文件夹下的文件都将被忽略。
    folders_to_exclude: Set[str] = {'__pycache__', '.venv', 'node_modules', '.git', 'dist', 'build', '__pycache__', 'WebUIAgentLogs'}
    # folders_to_exclude = None # 如果不想排除任何文件夹

    concatenate_files_with_separator(
        root_directory=source_directory,
        output_filepath=concatenated_output_file,
        target_extensions=include_extensions,
        excluded_folders=folders_to_exclude
    )

    # 简单的验证输出文件内容
    output_p = Path(concatenated_output_file)
    if output_p.exists() and output_p.stat().st_size > 0:
        print(f"\n--- '{concatenated_output_file}' 的前200个字符 ---")
        with open(output_p, 'r', encoding='utf-8') as f:
            print(f.read(200) + "...")
        print("--- END ---")
    elif output_p.exists():
        print(f"\n--- '{concatenated_output_file}' 是空的 ---")

开始处理目录: D:\ALL IN AI\IDTAgent\IDT_Agent_Native
输出文件: D:\ALL IN AI\IDTAgent\IDT_Agent_Native\completeCode.txt
目标扩展名: ['.txt', '.py', '.js', '.html', '.css']
排除的文件夹: {'.git', 'node_modules', '__pycache__', 'build', '.venv', 'dist', 'WebUIAgentLogs'}
  已处理: CircuitManusCore.py
  已处理: completeCode.txt
  已处理: requirements.txt
  已处理: server.py
  已处理: circuitmanus/agent.py
  已处理: circuitmanus/__init__.py
  已处理: circuitmanus/circuit_domain/circuit.py
  已处理: circuitmanus/circuit_domain/components.py
  已处理: circuitmanus/circuit_domain/__init__.py
  已处理: circuitmanus/llm/interface.py
  已处理: circuitmanus/llm/parser.py
  已处理: circuitmanus/llm/__init__.py
  已处理: circuitmanus/memory/manager.py
  已处理: circuitmanus/memory/__init__.py
  已处理: circuitmanus/prompts/templates.py
  已处理: circuitmanus/prompts/__init__.py
  已处理: circuitmanus/tools/base.py
  已处理: circuitmanus/tools/circuit_ops.py
  已处理: circuitmanus/tools/executor.py
  已处理: circuitmanus/tools/web_search.py
  已处理: circuitmanus/tools/__init__.py
 